## Karate Club

<img src="http://www-rohan.sdsu.edu/~gawron/python_for_ss/course_core/book_draft/_images/karate_club_circular.png"/>

Zachary’s karate club network: View 1 (Circular layout)

<img src="http://www-rohan.sdsu.edu/~gawron/python_for_ss/course_core/book_draft/_images/karate_club_spring.png"/>

Zachary’s karate club network: View 2 (Fruchterman-Reingold layout)

<img src="http://www-rohan.sdsu.edu/~gawron/python_for_ss/course_core/book_draft/_images/karate_factions.png"/>

Zachary’s karate club network: The 2 factions (using community detection)

In [1]:
import networkx as nx

kn=nx.karate_club_graph()
print("Node Degree")
for v in kn:
    print('%s %s' % (v,kn.degree(v)))

Node Degree
0 16
1 9
2 10
3 6
4 3
5 4
6 4
7 4
8 5
9 2
10 3
11 1
12 2
13 5
14 2
15 2
16 2
17 2
18 2
19 3
20 2
21 2
22 2
23 5
24 3
25 3
26 2
27 4
28 3
29 4
30 4
31 6
32 12
33 17


In [2]:
print("radius: %d" % nx.radius(kn)) # min graph eccentricity 
print("diameter: %d" % nx.diameter(kn)) # longest shortest path
print("eccentricity: %s" % nx.eccentricity(kn)) # max graph distance between node and other node of the graph
print("center: %s" % nx.center(kn)) # set of nodes with minimum eccentricity
print("periphery: %s" % nx.periphery(kn)) # subgraph induced by vertices that have graph eccentricities equal to the graph diameter
print("density: %s" % nx.density(kn)) # number of edges is close to the maximal number of edges

radius: 3
diameter: 5
eccentricity: {0: 3, 1: 3, 2: 3, 3: 3, 4: 4, 5: 4, 6: 4, 7: 4, 8: 3, 9: 4, 10: 4, 11: 4, 12: 4, 13: 3, 14: 5, 15: 5, 16: 5, 17: 4, 18: 5, 19: 3, 20: 5, 21: 4, 22: 5, 23: 5, 24: 4, 25: 4, 26: 5, 27: 4, 28: 4, 29: 5, 30: 4, 31: 3, 32: 4, 33: 4}
center: [0, 1, 2, 3, 8, 13, 19, 31]
periphery: [14, 15, 16, 18, 20, 22, 23, 26, 29]
density: 0.139037433155


In [3]:
# Betweenness Centrality: number of short paths a node is on

betweenness = nx.betweenness_centrality(kn)
sorted(betweenness.items(), key=lambda x:-x[1])[:10]

[(0, 0.4376352813852815),
 (33, 0.304074975949976),
 (32, 0.145247113997114),
 (2, 0.14365680615680618),
 (31, 0.13827561327561325),
 (8, 0.05592682780182781),
 (1, 0.053936688311688304),
 (13, 0.04586339586339586),
 (19, 0.03247504810004811),
 (5, 0.02998737373737374)]

In [4]:
# nodes 0 and 33 have highest betweenness scores - thought to be most central people in 
# this social organization. lets compare with degree centrality
# -> results aren't very different

deg = nx.degree_centrality(kn)
sorted(deg.items(), key=lambda x:-x[1])[:10]

[(33, 0.5151515151515151),
 (0, 0.48484848484848486),
 (32, 0.36363636363636365),
 (2, 0.30303030303030304),
 (1, 0.2727272727272727),
 (3, 0.18181818181818182),
 (31, 0.18181818181818182),
 (8, 0.15151515151515152),
 (13, 0.15151515151515152),
 (23, 0.15151515151515152)]

## Girvan-Newman Algorithm

In [ ]:
# Let's implement the algorithm (based off what we did for homework last week)

import networkx as nx

def find_best_edge(G0):
        """
        Networkx implementation of edge_betweenness
        returns a dictionary. Make this into a list,
        sort it and return the edge with highest betweenness.
        """
        eb = nx.edge_betweenness_centrality(G0)
        eb_il = eb.items()
        eb_il.sort(key=lambda x: x[1], reverse=True)
        return eb_il[0][0]

def girvan_newman(G):

    if len(G.nodes()) == 1:
        return [G.nodes()]

    components = list(nx.connected_component_subgraphs(G))
    while len(components) == 1:
        G.remove_edge(*find_best_edge(G))
        components = list(nx.connected_component_subgraphs(G))

    result = [c.nodes() for c in components]

    for c in components:
        result.extend(girvan_newman(c))

    return result

In [ ]:
gn = girvan_newman(kn)

In [ ]:
gn

In [ ]:
print gn[0]
print gn[1]

## Louvain Modularity

In [ ]:
# pip install python-louvain
from community import *

part = best_partition(kn) # mapping of node_id -> cluster_id
part

In [ ]:
# find modularity
part = best_partition(kn)
mod = modularity(part, kn)
print mod

In [ ]:
%pylab inline
# color maps here - http://matplotlib.org/examples/color/colormaps_reference.html

values = [part.get(node) for node in kn.nodes()]
values

In [ ]:
pos = nx.spring_layout(kn)
nx.draw(kn, pos, cmap = plt.get_cmap('jet'), node_color = values, node_size=800, with_labels=True)

In [ ]:
# create histogram of membership to modularity group
part_vals = sorted(set(part.values()))
part_hist = [part.values().count(x) for x in part_vals]

plot(part_vals, part_hist, 'ro-')
title('membership to cluster group')
xlabel('cluster group #')
ylabel('membership size')
rcParams['figure.figsize'] = 10,3


## Onto...  #NationalKaleDay